In [1]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings("ignore")

# DATA

In [3]:
app_train = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_train.csv')
app_test = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_test.csv')
prev_app = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\prev_app.csv')
pos_cash = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\pos_cash.csv')
bureau = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\bureau_balance_merge_bureau.csv')
ccb = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\credit_card_balance_final.csv')
inp = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\installments_payments_final.csv').drop(columns='Unnamed: 0')

In [4]:
inp.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'},inplace=True)
full_train = pd.merge(app_train, bureau, on= 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,prev_app, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,inp, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,ccb, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,pos_cash, on = 'SK_ID_CURR', how = 'left')

# Preprocessed

In [5]:
def find_high_nulL_per(data, threshold = 70):
    '''Func to drop col with %null > threshold (default 70)
        Input : data_train
    '''
    null_check = data.isnull().sum() / data.shape[0] * 100
    null_check = null_check.drop(null_check[null_check == 0].index).sort_values(ascending= False).reset_index()
    drop_null_col =null_check[null_check[0] > threshold]['index'].tolist()
    return drop_null_col


In [6]:
def check_values(column):
    # Check for infinity
    has_infinity = np.any(np.isinf(column))
    
    # Check for values too large for float64
    too_large_values = np.any(np.abs(column) > np.finfo(np.float64).max)
    
    return has_infinity, too_large_values

# Iterate over columns and check for infinity and large values
def find_col(data):
    cols = []
    for col in data.columns:
        has_infinity, too_large_values = check_values(data[col])
        
        if has_infinity:
            print(f"Column '{col}' contains infinity.")
            cols.append(col)

        
        if too_large_values:
            print(f"Column '{col}' contains values too large for dtype('float64').")
            if col not in cols:
                cols.append(col)
    return cols


In [7]:
def fill_nan(data):
    '''Input is X_ord_trained
        Output is all the things that train do and they is used for test
    '''
    # Check Infinity and Replace them to Nan

    # Fill nan
    ### Numeric data
    num_data = data._get_numeric_data().drop(columns = ['SK_ID_CURR','TARGET'])
    mean_num_data = num_data.mean()
    ### Categorical data
    cat_data = data[[col for col in data.columns if col not in num_data.columns and col not in ['SK_ID_CURR','TARGET']]]
    mode_cat_data = cat_data.mode().iloc[0]


    return mean_num_data,num_data.columns ,mode_cat_data, cat_data.columns
    

In [8]:
full_train.drop(columns = find_high_nulL_per(full_train),axis = 1,inplace=True)
app_test.drop(columns = find_high_nulL_per(full_train),axis = 1, inplace=True)

# Join both train and test

In [9]:
full_app = pd.concat([app_train, app_test], axis = 0)

In [10]:
full_data = pd.merge(full_app, bureau, on= 'SK_ID_CURR', how = 'left')
full_data = pd.merge(full_data,prev_app, on = 'SK_ID_CURR', how = 'left')
full_data = pd.merge(full_data,inp, on = 'SK_ID_CURR', how = 'left')
full_data = pd.merge(full_data,ccb, on = 'SK_ID_CURR', how = 'left')
full_data = pd.merge(full_data,pos_cash, on = 'SK_ID_CURR', how = 'left')

In [11]:
# handle infinity
inf_cols = find_col(full_data._get_numeric_data())
# replace inf to Nan
for i in inf_cols:
    full_data[i] = full_data[i].replace(np.inf, np.nan)
    full_train[i] = full_train[i].replace(np.inf, np.nan)

# check again
print("\nColumns with Infinity Values After Replacement (full_data):")
inf_cols1 = find_col(full_data._get_numeric_data())
print(inf_cols1)

inf_cols2 = find_col(full_train._get_numeric_data())
print("\nColumns with Infinity Values After Replacement (full_train):")
print(inf_cols2)


Column 'CAR_EMPLOYED_RATIO' contains infinity.
Column 'CAR_EMPLOYED_RATIO' contains values too large for dtype('float64').
Column 'PAYS_TO_INS_' contains infinity.
Column 'PAYS_TO_INS_' contains values too large for dtype('float64').

Columns with Infinity Values After Replacement (full_data):
[]

Columns with Infinity Values After Replacement (full_train):
[]


In [12]:
full_data['Total_nan'] = full_data.isna().sum(axis= 1)

fill_num,num_col ,fill_cat,cat_col = fill_nan(full_train)
full_data[num_col] = full_data[num_col].fillna(fill_num)
full_data[cat_col] = full_data[cat_col].fillna(fill_cat)
full_train[num_col] = full_train[num_col].fillna(fill_num)
full_train[cat_col] = full_train[cat_col].fillna(fill_cat)

# Transformation

In [13]:
for i in cat_col:
    print(f'{i}  -  {full_train[i].unique()}')

NAME_CONTRACT_TYPE  -  ['Cash loans' 'Revolving loans']
CODE_GENDER  -  ['F' 'M' 'XNA']
FLAG_OWN_CAR  -  ['N' 'Y']
FLAG_OWN_REALTY  -  ['N' 'Y']
NAME_TYPE_SUITE  -  ['Family' 'Unaccompanied' 'Spouse, partner' 'Children' 'Other_A'
 'Group of people' 'Other_B']
NAME_INCOME_TYPE  -  ['State servant' 'Working' 'Pensioner' 'Commercial associate' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']
NAME_EDUCATION_TYPE  -  ['Higher education' 'Secondary / secondary special' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
NAME_FAMILY_STATUS  -  ['Married' 'Civil marriage' 'Single / not married' 'Widow' 'Separated'
 'Unknown']
NAME_HOUSING_TYPE  -  ['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']
OCCUPATION_TYPE  -  ['Core staff' 'Laborers' 'Managers' 'Drivers' 'Sales staff'
 'Cleaning staff' 'Private service staff' 'Medicine staff'
 'Security staff' 'Accountants' 'Cooking staff' 'High skill tech staff'
 'Low-skill

In [14]:
# ordinal cols
ord_col = ['INS_ON_TIME_GRADE_','PAYMENT_GRADE_','FLAG_CONTACT']
X_ord = full_data[ord_col]

ins_on_time_grade_encode = {'Good': 0, 'Par' : 1, 'Sub_Par' : 2, 'Poor' : 3,'Very_Poor': 4}
X_ord['INS_ON_TIME_GRADE_'] = X_ord['INS_ON_TIME_GRADE_'].map(ins_on_time_grade_encode)

ins_on_time_grade_encode = {'Good': 0, 'Par' : 1, 'Sub_Par' : 2, 'Poor' : 3,'Very_Poor': 4}
X_ord['PAYMENT_GRADE_'] = X_ord['PAYMENT_GRADE_'].map(ins_on_time_grade_encode)

ins_on_time_grade_encode = {'Good': 0, 'Normal' : 1, 'Bad' : 2}
X_ord['FLAG_CONTACT'] = X_ord['FLAG_CONTACT'].map(ins_on_time_grade_encode)




In [15]:
# norminal cols
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
X_one_hot = full_train[[col for col in cat_col if col not in ord_col]]
X_one_hot = one_hot_encoder.fit(X_one_hot)


# Fit and transform on full_train
X_one_hot_train = one_hot_encoder.fit_transform(full_train[[col for col in cat_col if col not in ord_col]])
X_onehot_data = one_hot_encoder.transform(full_data[[col for col in cat_col if col not in ord_col]])
# Convert to array and concatenate with other features

X_onehot_data = X_onehot_data.toarray()
X_cat_data = pd.DataFrame(X_onehot_data)
X_cat = pd.concat([X_ord, X_cat_data], axis=1)

In [16]:
find_col(full_data[num_col])

[]

In [17]:
from sklearn.preprocessing import StandardScaler

X_num = full_train[num_col]
scaler = StandardScaler()
# fit with X_train
X_num = scaler.fit(X_num)


# Transform the full_data
X_num = pd.DataFrame(scaler.transform(full_data[num_col]))


In [18]:
full_set = pd.concat([X_num,X_cat],axis = 1)
full_set.columns = [i for i in range(full_set.shape[1])]
full_set['SK_ID_CURR'] = full_data['SK_ID_CURR']
full_set['TARGET'] = full_data['TARGET']


In [19]:
full_set

,0,1,2,3,4,5,6,7,8,9,...,356,357,358,359,360,361,362,363,SK_ID_CURR,TARGET
0,-0.577564,0.971411,1.721325,0.591184,1.596499,-1.252985,0.130494,-0.521152,1.078900,1.790480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,278621,0.0
1,-0.577564,-0.320090,-0.711884,0.176858,-0.654215,-0.928918,0.716395,0.339087,-1.376668,0.368420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139008,0.0
2,-0.577564,-0.449240,-0.214938,-0.362138,-0.070246,0.565060,0.883795,0.339087,0.191465,-0.308152,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138348,0.0
3,-0.577564,-0.664491,-0.270767,0.027378,-0.228404,1.080977,0.214194,-0.349105,0.004323,1.667226,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64140,0.0
4,-0.577564,1.832412,2.308026,1.030630,2.679274,-1.283307,0.632695,-0.865248,0.110247,0.406854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,219374,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,-0.577564,0.024310,-0.194397,-0.277784,-0.240570,-0.160725,1.804497,-0.005009,0.342826,-1.651356,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,150442,NaN
307507,2.197660,2.693414,0.745128,1.232211,0.976032,-1.252985,0.632695,-0.177057,-0.099330,0.556614,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5217,NaN
307508,-0.577564,0.540911,-0.985224,-0.212968,-0.909701,0.565060,-0.873909,-1.037296,1.076060,-1.010568,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,260741,NaN
307509,-0.577564,-0.449240,-0.855727,0.223066,-0.848871,-1.232939,1.720797,-0.005009,0.740965,-0.839602,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,284794,NaN


# SPLIT

In [20]:
train = full_set[(full_set['TARGET'] == 0) | (full_set['TARGET'] == 1)]
test = full_set[~((full_set['TARGET'] == 0) | (full_set['TARGET'] == 1))]

In [21]:
#Select KBest
from sklearn.feature_selection import f_classif,SelectKBest
x_train = train.drop(columns= 'TARGET').set_index('SK_ID_CURR')
y_train = train['TARGET']
selected = SelectKBest(f_classif, k=int((train.shape[1] * 0.8)))
X_new = selected.fit_transform(x_train, y_train)
selected_col = selected.get_support(indices=True)

In [22]:
train = train[[col for col in selected_col]]
train['TARGET'] = y_train
train['SK_ID_CURR'] = x_train.index
train = train.set_index('SK_ID_CURR')
train

,0,1,2,3,4,5,6,7,8,9,...,348,352,353,356,357,360,361,362,363,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
278621,-0.577564,0.971411,1.721325,0.591184,1.596499,-1.252985,0.130494,-0.521152,1.078900,1.790480,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139008,-0.577564,-0.320090,-0.711884,0.176858,-0.654215,-0.928918,0.716395,0.339087,-1.376668,0.368420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138348,-0.577564,-0.449240,-0.214938,-0.362138,-0.070246,0.565060,0.883795,0.339087,0.191465,-0.308152,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64140,-0.577564,-0.664491,-0.270767,0.027378,-0.228404,1.080977,0.214194,-0.349105,0.004323,1.667226,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219374,-0.577564,1.832412,2.308026,1.030630,2.679274,-1.283307,0.632695,-0.865248,0.110247,0.406854,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90253,-0.577564,-0.104840,-0.855727,0.030169,-0.848871,0.847153,-1.543510,-1.037296,-0.985629,0.669929,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9027,-0.577564,-0.922791,-0.818887,-1.041931,-0.848871,0.311843,1.051196,-0.005009,0.169599,-0.726951,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
163598,-0.577564,-0.147890,0.193559,0.197016,0.124410,-1.147254,-0.204307,2.575709,-0.497469,-1.429366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
test = test.set_index('SK_ID_CURR')
test = test[selected_col]
test

,0,1,2,3,4,5,6,7,8,9,...,347,348,352,353,356,357,360,361,362,363
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
83659,2.197660,0.368711,-0.332881,1.758802,-0.325732,-0.812332,-0.622808,-0.693200,1.234805,-0.870747,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174814,-0.577564,0.756161,1.692077,1.504190,1.730325,-1.013372,-0.288007,-0.521152,0.958493,-1.029122,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179486,-0.577564,0.325661,-0.259592,0.826258,-0.118910,1.080977,0.465294,-0.865248,0.703764,1.015173,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57038,-0.577564,0.756161,-0.855727,-0.150323,-0.848871,1.835637,0.800095,2.231613,-1.755497,-0.109355,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25672,-0.577564,-0.535341,-0.723170,-0.775844,-0.824539,-0.145093,0.967495,-0.521152,-0.657065,1.655961,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150442,-0.577564,0.024310,-0.194397,-0.277784,-0.240570,-0.160725,1.804497,-0.005009,0.342826,-1.651356,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5217,2.197660,2.693414,0.745128,1.232211,0.976032,-1.252985,0.632695,-0.177057,-0.099330,0.556614,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
260741,-0.577564,0.540911,-0.985224,-0.212968,-0.909701,0.565060,-0.873909,-1.037296,1.076060,-1.010568,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
train.to_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\train.csv')
test.to_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\test.csv')